⚠️ This notebook relies on version 1 of the fastai library. If Google Colab or your local python environment upgrade to version 2, it will no longer work 

# Twelve way segmentation workbook
- Takes an input data volume and a 2D Unet trained for binary segmentation - our trained models and some test data can be downloaded from Zenodo [![DOI](https://zenodo.org/badge/DOI/10.5281/zenodo.4249627.svg)](https://doi.org/10.5281/zenodo.4249627)
- Slices the data volume in the three orthogonal planes and predicts output for each slice
- The predictions are recombined into 3D volumes and then summed
- The input data volume is rotated by 90 degrees before the slicing and prediction steps are performed again
- This is repeated until 4 rotations have been been performed
- All the volumes are summed to give a prediction that is the sum of predictions in 12 different directions, a list of threshold values for a consensus cutoff is used to give a number of output volumes

### Important for Colab use
If running this notebook on Google Colab, please go to the 'runtime' menu then 'change runtime type' and select 'GPU' as the 'Hardware Accelerator'

In [ ]:
import os
from datetime import date
import re
import numpy as np
import dask.array as da
import h5py as h5
from fastai.vision import *
from skimage import img_as_ubyte, io, exposure, img_as_float
from skimage.transform import resize
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

In [ ]:
# If using Google Colab, run this cell to download the Unet and the data for prediction from Zenodo
!mkdir -p prediction_data/
!curl  -o prediction_data/specimen1_512cube_zyx_800-1312_1000-1512_700-1212_DATA.h5 https://zenodo.org/api/files/fc8e12d1-4256-4ed9-8a23-66c0d6c64379/specimen1_512cube_zyx_800-1312_1000-1512_700-1212_DATA.h5
!curl  -o prediction_data/specimen1_placental_blood_vessels_2dUnet.pkl https://zenodo.org/api/files/fc8e12d1-4256-4ed9-8a23-66c0d6c64379/specimen1_placental_blood_vessels_2dUnet.pkl

### Setup Paths that are needed
1. `root_path` - Root filepath for output directories, folder will be created
2. `data_vol_path` - Path to the HDF5 volume to be segmented. Data should be in `/data` inside the file
3. `learner_root_path` - Path to the folder containing the model file
4. `learner_file` - Filename of the pickled 2d Unet model file. Needs to have been trained using BCE loss. For binary segmentation only
5. `consensus_vals` - List of consensus cutoff values for agreement between volumes e.g. if 10 is in the list a volume will be output thresholded on consensus between 10 volumes

In [ ]:
root_path = Path('data_output')
data_vol_path = Path('prediction_data/specimen1_512cube_zyx_800-1312_1000-1512_700-1212_DATA.h5')
learner_root_path = Path('prediction_data')
learner_file = 'specimen1_placental_blood_vessels_2dUnet.pkl'
consensus_vals = [8, 9]

### Utility functions

In [ ]:
makedirs = partial(os.makedirs, exist_ok=True)

def da_from_data(path):
    """Returns a dask array when given a path to an HDF5 file.
    
    The data is assumed to be found in '/data' in the file.
    
    Args:
        path (Path): The path to the HDF5 file.

    Returns:
        dask.array: A dask array object for the data stored in the HDF5 file."""

    f = h5.File(path, 'r')
    d = f['/data']
    return da.from_array(d, chunks='auto')

# Needed because prediction doesn't work on odd sized images
def fix_odd_sides(example_image):
    """Replaces an an odd image dimension with an even dimension by padding.
    
    Taken from https://forums.fast.ai/t/segmentation-mask-prediction-on-different-input-image-sizes/44389/7."""
    if (list(example_image.size)[0] % 2) != 0:
        example_image = crop_pad(example_image, 
                            size=(list(example_image.size)[0]+1, list(example_image.size)[1]),
                            padding_mode = 'reflection')

    if (list(example_image.size)[1] % 2) != 0:
        example_image = crop_pad(example_image, 
                            size=(list(example_image.size)[0], list(example_image.size)[1] + 1),
                            padding_mode = 'reflection')

def predict_single_slice(learn, axis, val, data, output_path):
    """Takes in a 2d data array and saves the predicted U-net segmentation to disk.
    
    Args:
        learn (fastai.vision.learner): The trained 2d U-net model.
        axis (str): The name of the axis to incorporate in the output filename.
        val (int): The slice number to incorporate in the output filename.
        data (numpy.array): The 2d data array to be fed into the U-net.
        output_path (Path): The path to directory for file output."""
    data = img_as_float(data)
    img = Image(pil2tensor(data, dtype=np.float32))
    fix_odd_sides(img)
    prediction = learn.predict(img)
    pred_slice = img_as_ubyte(prediction[1][0])
    io.imsave(output_path/f"unet_prediction_{axis}_stack_{val}.png", pred_slice)

def predict_orthog_slices_to_disk(learn, axis, data_arr, output_path):
    """Coordinates the 2d U-net prediction of an image volume in one or all of the three orthogonal
    planes to images on disk. 
    
    Args:
        learn (fastai.vision.learner): The trained 2d U-net model.
        axis (string): Which plane to slice the data in. Either 'x', 'y, 'z' or 'all'.
        data_array (array): The data volume to be sliced and predicted.
        output_path (Path): A Path object to the output directory."""
    data_shape = data_arr.shape
    name_prefix = 'seg'
    # There has to be a cleverer way to do this!
    if axis in ['z', 'all']:
        print('Predicting z stack')
        for val in range(data_shape[0]):
            predict_single_slice(learn, 'z', val, data_arr[val, :, :], output_path)
    if axis in ['x', 'all']:
        print('Predicting x stack')
        for val in range(data_shape[1]):
            predict_single_slice(learn, 'x', val, data_arr[:, val, :], output_path)                    
    if axis in ['y', 'all']:
        print('Predicting y stack')
        for val in range(data_shape[2]):
            predict_single_slice(learn, 'y', val, data_arr[:, :, val], output_path)
    if axis not in ['x', 'y', 'z', 'all']:
        print("Axis should be one of: [all, x, y, or z]!")

def setup_folder_stucture(root_path):
    """Sets up a folder structure to store the predicted images.
    
    Args:
        root_path (Path): The top level directory for data output.

    Returns:
        list: A list of tuples containing a key and the corresponding path for a folder."""
    non_rotated = root_path/f'{date.today()}_non_rotated_seg_slices'
    rot_90_seg = root_path/f'{date.today()}_rot_90_seg_slices'
    rot_180_seg = root_path/f'{date.today()}_rot_180_seg_slices'
    rot_270_seg = root_path/f'{date.today()}_rot_270_seg_slices'
    
    dir_list = [
        ('non_rotated', non_rotated),
        ('rot_90_seg', rot_90_seg),
        ('rot_180_seg', rot_180_seg),
        ('rot_270_seg', rot_270_seg)
    ]
    for key, dir_path in dir_list:
        makedirs(dir_path)
    return dir_list

# Need the loss in order to load the learner..
def bce_loss(logits, labels):
    """Defines the binary cross entropy loss function used when training a U-net on binary data."""
    logits=logits[:,1,:,:].float()
    labels = labels.squeeze(1).float()
    return F.binary_cross_entropy_with_logits(logits, labels)

# Since we're doing binary segmentation, we need to define a binary label list class
class BinaryLabelList(SegmentationLabelList):
    def open(self, fn): return open_mask(fn)

class BinaryItemList(SegmentationItemList):
    _label_cls = BinaryLabelList

def combine_slices_to_vol(folder_path):
    """Combines the orthogonally sliced png images in a folder to HDF5 volumes with a common orientation."""
    output_path_list = []
    file_list = folder_path.ls()
    axis_list = ['z', 'x', 'y']
    axis_regex = re.compile(r'\_(\D)\_')
    number_regex = re.compile(r'\_(\d+)\.png')
    for axis in axis_list:
        axis_files = [x for x in file_list if re.search(f'\_({axis})\_', str(x))]
        print(f"Creating volume from {axis} stack")
        print(f'{len(axis_files)} files found')
        first_im = open_image(axis_files[0])
        shape_tuple = first_im.shape
        z_dim = len(axis_files)
        x_dim = shape_tuple[1]
        y_dim = shape_tuple[2]
        data_vol = np.empty([z_dim, x_dim, y_dim], dtype=np.uint8)
        for filename in axis_files:
            m = number_regex.search(str(filename))
            pos = int(m.group(1))
            im_data = io.imread(filename)
            data_vol[pos, :, :] = im_data
        if axis == 'x':
            data_vol = np.swapaxes(data_vol, 0, 1)
        if axis == 'y':
            data_vol = np.swapaxes(data_vol, 0, 2)
            data_vol = np.swapaxes(data_vol, 0, 1)
        output_path = folder_path/f'{axis}_axis_seg_combined.h5'
        output_path_list.append(output_path)
        print(f'Outputting volume to {output_path}')
        with h5.File(output_path, 'w') as f:
            f['/data'] = data_vol
        # Delete the images
        print(f"Deleting {len(axis_files)} image files for axis {axis}")
        for filename in axis_files:
            os.remove(filename)
    return output_path_list

def combine_vols(output_path_list, k, prefix, final=False):
    """Sums volumes to give a combination of binary segmentations and saves to disk."""
    num_vols = len(output_path_list)
    combined = da_from_data(output_path_list[0])
    for subsequent in output_path_list[1:]:
        combined += da_from_data(subsequent)
    combined_out_path = output_path_list[0].parent.parent/f'{date.today()}_{prefix}_{num_vols}_volumes_combined.h5'
    if final:
        combined_out_path = output_path_list[0].parent/f'{date.today()}_{prefix}_{num_vols}_volumes_combined.h5'
    print(f'Outputting the {num_vols} combined volumes to {combined_out_path}')
    combined = combined.compute()
    combined = np.rot90(combined, k, (1, 0))
    with h5.File(combined_out_path, 'w') as f:
        f['/data'] = combined
    return combined_out_path

def threshold(input_path, range_list):
    """Outputs a consensus thresholded volume from combination of binary volumes."""
    for val in range_list:
        combined = da_from_data(input_path)
        combined_out = input_path.parent/f'{date.today()}_combined_thresh_cutoff_{val}.h5'
        combined[combined < val] = 0
        combined[combined >= val] = 255
        print(f'Writing to {combined_out}')
        combined.to_hdf5(combined_out, '/data')

### Make a root directory for the output

In [ ]:
makedirs(root_path)

### Load the data volume and the model

In [ ]:
data_arr = da_from_data(data_vol_path)
data_arr

In [ ]:
learn = load_learner(learner_root_path, learner_file)
# Remove the restriction on the model prediction size
learn.data.single_ds.tfmargs['size'] = None

### Run the loop to do repeated prediction and recombination steps 

The final combined output file of 12 summed volumes will be found in the `data_output` directory with a filename `YYYY-MM_DD_final_4_volumes_combined` and the files with a consensus threshold applied will be found in file with filenames `YYYY-MM_DD_combined_thresh_cutoff_X`

In [ ]:
axis = 'all'
dir_list = setup_folder_stucture(root_path)
combined_vol_paths = []
data_arr = data_arr.compute()
for k in range(4):
    key, output_path = dir_list[k]
    print(f'Key : {key}, output : {output_path}')
    print(f'Rotating volume {k * 90} degrees')
    rotated = np.rot90(data_arr, k)
    predict_orthog_slices_to_disk(learn, axis, rotated, output_path)
    output_path_list = combine_slices_to_vol(output_path)
    fp = combine_vols(output_path_list, k, key)
    combined_vol_paths.append(fp)
# Combine all the volumes
final_combined = combine_vols(combined_vol_paths, 0, 'final', True)
threshold(final_combined, consensus_vals)